In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [2]:
#Import training set
df = pd.read_csv(r'/kaggle/input/playground-series-s4e8/train.csv')

#Look at head of data
df.head(10)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
5,5,p,4.30,x,t,n,f,s,c,n,...,NaN,NaN,w,NaN,n,t,z,NaN,d,a
6,6,e,9.65,p,y,w,f,e,c,k,...,NaN,s,w,NaN,NaN,t,e,NaN,g,w
7,7,p,4.55,x,e,e,f,a,NaN,y,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
8,8,p,7.36,f,h,e,f,x,d,w,...,b,NaN,w,NaN,NaN,f,f,NaN,d,a
9,9,e,6.45,x,t,n,f,a,d,w,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,a


In [3]:
#Let's also get a look at the columns themselves
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-root             object 
 13  stem-surface          object 
 14  stem-color            object 
 15  veil-type             object 
 16  veil-color            object 
 17  has-ring              object 
 18  ring-type             object 
 19  spore-print-color     object 
 20  habitat               object 
 21  season                object 
dtypes: float64(3), int64(1), object(18)
memory

In [4]:
#There appears to be many missing values in some of the columns, lets get null value counts
df.isnull().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [5]:
#Looks like a few columns are missing quite a few values.
#Lets first drop all nulls and see what we're left with. We'll create a new dataframe so we don't mess up the data.
df_drop_na = df.dropna()
df_drop_na.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season


In [6]:
#Obviously that didn't work. Let's try a different approach. 
#Let's create a new datafram where we drop all the columns that are missing more than 500000 points.
cols_to_drop = [col for col in df.columns
                if df[col].isnull().sum() > 500000]

df_dropped = df.drop(cols_to_drop, axis=1)
df_dropped.isnull().sum()

id                           0
class                        0
cap-diameter                 4
cap-shape                   40
cap-color                   12
does-bruise-or-bleed         8
gill-color                  57
stem-height                  0
stem-width                   0
stem-color                  38
has-ring                    24
ring-type               128880
habitat                     45
season                       0
dtype: int64

In [7]:
#Let's look at some individual columns for a second. For the 'does-bruise-or-bleed' column, we should only have false or true as the entries. However...
df_dropped['does-bruise-or-bleed'].value_counts()

does-bruise-or-bleed
f           2569743
t            547085
w                14
c                11
h                 9
a                 7
b                 7
x                 7
y                 7
k                 6
s                 6
e                 4
l                 4
p                 4
d                 4
g                 3
z                 3
o                 3
i                 2
n                 2
r                 1
has-ring          1
3.43              1
4.42              1
2.9               1
u                 1
Name: count, dtype: int64

In [8]:
#Let's look at 'ring-type' column as well, which has the most missing values right now.
df_dropped['ring-type'].value_counts()

ring-type
f                       2477170
e                        120006
z                        113780
l                         73443
r                         67909
p                         67678
g                         63687
m                          3992
t                            98
d                            37
n                            34
x                            32
b                            29
y                            25
s                            18
k                            18
a                            16
h                            15
w                            14
u                            13
c                            11
o                             9
ring-type                     7
i                             4
does f                        3
season                        3
15                            1
14                            1
2.87                          1
sp                            1
class                         

In [9]:
# It seems that every column is littered with these random values which clutter the data. Let's clean it up by removing any values that appear less than 1000 times in any column.
df_cleaned = df_dropped.copy()

for col in df_cleaned.columns:
    if df_cleaned[col].dtype != 'object':
        continue
    else:
        counts = df_cleaned[col].value_counts()
        valids = counts[counts > 1000].index
        df_cleaned = df_cleaned[df_cleaned[col].isin(valids)]

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2984742 entries, 0 to 3116944
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-color             object 
 5   does-bruise-or-bleed  object 
 6   gill-color            object 
 7   stem-height           float64
 8   stem-width            float64
 9   stem-color            object 
 10  has-ring              object 
 11  ring-type             object 
 12  habitat               object 
 13  season                object 
dtypes: float64(3), int64(1), object(10)
memory usage: 341.6+ MB


In [10]:
# Finally we drop any remaining nulls to remove the issues with the float variables and finalize our clean dataset.
df_cleaned = df_cleaned.dropna()
df_cleaned.isnull().sum()

id                      0
class                   0
cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64

In [11]:
df_cleaned = df_cleaned.drop('id', axis=1).reset_index(drop=True)
df_cleaned.head()

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,e,8.80,f,u,f,w,4.51,15.39,w,f,f,d,a
1,p,4.51,x,o,f,n,4.79,6.48,o,t,z,d,w
2,e,6.94,f,b,f,w,6.85,9.93,n,f,f,l,w
3,e,3.88,f,g,f,g,4.16,6.53,w,f,f,d,u
4,e,5.85,x,w,f,w,3.37,8.36,w,f,f,g,a


In [12]:
# Import modules from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#Setup data with train/test splits
Y = df_cleaned['class'].copy()
X = df_cleaned.drop('class', axis=1)
X = pd.get_dummies(X, drop_first=True, dtype=float)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [13]:
#Create our initial random forest model
forest = RandomForestClassifier(n_estimators = 10)

In [14]:
# Fit model (note, was not initally fit because I was messing with number of trees so it didn't take an hour to run)
forest.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=10)

In [15]:
# Print score on testing set
forest.score(X_test, Y_test)

0.9812200280984832

In [16]:
# Looks good for an initial submission! Let's recreate and fit the model to the entire X set, then create a submission for the competition.
forest = RandomForestClassifier(n_estimators = 50).fit(X, Y)

In [17]:
# Import test data and clean it
test_data = pd.read_csv(r'/kaggle/input/playground-series-s4e8/test.csv', index_col='id')
test_data.isnull().sum()

cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [18]:
#Drop columns that we removed in the training data
test_data = test_data.drop(cols_to_drop, axis=1)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2077964 entries, 3116945 to 5194908
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   cap-diameter          float64
 1   cap-shape             object 
 2   cap-color             object 
 3   does-bruise-or-bleed  object 
 4   gill-color            object 
 5   stem-height           float64
 6   stem-width            float64
 7   stem-color            object 
 8   has-ring              object 
 9   ring-type             object 
 10  habitat               object 
 11  season                object 
dtypes: float64(3), object(9)
memory usage: 206.1+ MB


In [19]:
# We need to clean the data, but due to how submissions work we obviously can't just drop rows like we did in the training set.
# Here we impute the data.
test_cleaned = test_data.copy()

for col in test_cleaned.columns:
    #Replace nulls in numeric data with the mean
    if test_cleaned[col].dtype != 'object':
        col_mean = test_cleaned[col].mean()
        test_cleaned[col] = test_cleaned[col].fillna(col_mean)
        
    #For categorical variables, first identify which values appear too few times to be considered relevant/accurate.
    else:
        counts = test_cleaned[col].value_counts()
        valids = counts[counts < 1000].index
        # Now identify which value appears most
        most_common = test_cleaned[col].value_counts().idxmax()
        # Replace all irrelevant values with the value that appears most
        test_cleaned[col] = test_cleaned[col].replace(valids, most_common)
        # Finally, replace NaN with mode as well
        test_cleaned[col] = test_cleaned[col].fillna(most_common)

In [20]:
# Double check that data makes sense
print('Shape of intial test data:', test_data.shape)
print('Shape of cleaned data:', test_cleaned.shape)
test_cleaned.isnull().sum()

Shape of intial test data: (2077964, 12)
Shape of cleaned data: (2077964, 12)


cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64

In [21]:
# No null values, and no rows were lost, so we should be good to proceed! Let's get our dummy variables and run the predictions.
x_test = pd.get_dummies(test_cleaned, drop_first = True, dtype = float)
y_hat = forest.predict(x_test)

In [22]:
# Finally, we output to a csv file and submit!
output = pd.DataFrame({'id': x_test.index, 'class': y_hat})
output.to_csv('submission.csv', index=False)

Accuracy Score for the submission was: 0.95960. Not great, but not terrible. For a first solo competition I'll take it!